# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [35]:
import pandas as pd

## imports for Python, Pandas

In [36]:
import json
from pandas.io.json import json_normalize

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Solution to part 1: Find 10 Countries with Most Projects

In [37]:
# read json as string
Global_Projects_json_str = json.load((open('data/world_bank_projects.json')))

In [38]:
# examine normalize to further populate tables created from nested element
json_normalizedTable = json_normalize(Global_Projects_json_str)

In [39]:
# examine column names
print(json_normalizedTable.shape)
print(json_normalizedTable.columns)

(500, 55)
Index(['_id.$oid', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract.cdata', 'project_name',
       'projectdocs', 'projectfinancialtype', 'projectstatusdisplay',
       'regionname', 'sector', 'sector1.Name', 'sector1.Percent',
       'sector2.Name', 'sector2.Percent', 'sector3.Name', 'sector3.Percent',
       'sector4.Name', 'sector4.Percent', 'sector_namecode', 'sectorcode',
       'source', 'status', 'supplementprojectflg', 'theme1.Name',
       'theme1.Percent', 'theme_namecode', 'themecode', 'totalamt',
       'total

In [40]:
# load json file as Pandas dataframe
Global_Projects_json_df = pd.read_json('data/world_bank_projects.json')

In [41]:
# examine table size
Global_Projects_json_df.shape

(500, 50)

In [42]:
# examine 50 column names
print(Global_Projects_json_df.columns)

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')


In [43]:
#Difference in column names between json normalize and Data Frame
print(pd.Index(json_normalize(Global_Projects_json_str).columns).difference(pd.Index(Global_Projects_json_df.columns)))

Index(['_id.$oid', 'project_abstract.cdata', 'sector1.Name', 'sector1.Percent',
       'sector2.Name', 'sector2.Percent', 'sector3.Name', 'sector3.Percent',
       'sector4.Name', 'sector4.Percent', 'theme1.Name', 'theme1.Percent'],
      dtype='object')


In [44]:
# groupby country and count number of occurances based on 'project name' column
# sort the list in decending order
# save list of top ten countries with most number of projects
Projects_TopTen_Countries = Global_Projects_json_df.groupby('countryname')['project_name']\
                    .count().sort_values(ascending=False).head(10)

### Top 10 countries with most projects

In [45]:
Projects_TopTen_Countries.rename('Project Count', inplace=True)
Projects_TopTen_Countries.rename_axis('Country Name', inplace=True)
print(Projects_TopTen_Countries)

Country Name
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: Project Count, dtype: int64


### Solution to part 2: Find the top 10 major project themes (using column 'mjtheme_namecode')

In [46]:
# print(Global_Projects_json_df['mjtheme_namecode'])
# Extract project theme from major project list in column 'mjtheme_namecode'
# to create a table 'code' and 'name' of project theme
projList = []
for c_proj in Global_Projects_json_df['mjtheme_namecode']:
    for dict_item in c_proj:
        projList.append(dict_item.values())     #list(dict_item.values()))
MajorProjects = pd.DataFrame(projList, columns=['code','name']).astype(str)
print(MajorProjects.head(20)) 

   code                                          name
0     8                             Human development
1    11                                              
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion
10    5                         Trade and integration
11    4      Financial and private sector development
12    6         Social protection and risk management
13    6                                              
14    2                      Public sector governance
15    4      Financial and private sector development
16   11  Environment and natural resources management
17    8                     

In [47]:
# no missing values in 'code' column, thus, groupby 'code' and get list of top ten project theme codes
CodeCountRank = MajorProjects.groupby('code')['name'].count().sort_values(ascending=False)
CodeNameLevels = MajorProjects.groupby('name')['code'].value_counts()

LevelRowIndex = 11  # start at index 11 to remove blank names
CodeNameIndexLevel = list(CodeNameLevels.index.get_level_values(0)[LevelRowIndex:])
CodeIndexLevel = list(CodeNameLevels.index.get_level_values(1)[LevelRowIndex:])     
CodeNameLink = pd.DataFrame([CodeIndexLevel, CodeNameIndexLevel]).transpose()
print(CodeNameLink)

     0                                             1
0    1                           Economic management
1   11  Environment and natural resources management
2    4      Financial and private sector development
3    8                             Human development
4    2                      Public sector governance
5    3                                   Rule of law
6   10                             Rural development
7    7                   Social dev/gender/inclusion
8    6         Social protection and risk management
9    5                         Trade and integration
10   9                             Urban development


In [48]:
# Rank top tem projects
mjthemeTopTenList = []
for c_idx in CodeCountRank.index:
    boolArray = (CodeNameLink[0] == c_idx).astype(int)
    i=0
    for x in boolArray:
        if x == 1:
            break
        else:
            i =i+1
    matchCodeIndex = CodeNameLink[1][i]
    mjthemeTopTenList.append([CodeCountRank[c_idx], c_idx, CodeNameLink[1][i]]) 

### Top 10 Major Project Themes

In [49]:
mjTopProjects = pd.DataFrame(mjthemeTopTenList, columns = ('proj count','code', 'name'))
print(mjTopProjects)

    proj count code                                          name
0          250   11  Environment and natural resources management
1          216   10                             Rural development
2          210    8                             Human development
3          199    2                      Public sector governance
4          168    6         Social protection and risk management
5          146    4      Financial and private sector development
6          130    7                   Social dev/gender/inclusion
7           77    5                         Trade and integration
8           50    9                             Urban development
9           38    1                           Economic management
10          15    3                                   Rule of law


### Solution to part 3: Create a DataFrame with the Missing Names Filled.

In [50]:
print(MajorProjects.head(20)) 

   code                                          name
0     8                             Human development
1    11                                              
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion
10    5                         Trade and integration
11    4      Financial and private sector development
12    6         Social protection and risk management
13    6                                              
14    2                      Public sector governance
15    4      Financial and private sector development
16   11  Environment and natural resources management
17    8                     

In [51]:
for row in MajorProjects.values:
    if row[1] == '':
        row[1] = mjTopProjects['name'][mjTopProjects['code'] == row[0]].values[0]
print(MajorProjects.head(20))

   code                                          name
0     8                             Human development
1    11  Environment and natural resources management
2     1                           Economic management
3     6         Social protection and risk management
4     5                         Trade and integration
5     2                      Public sector governance
6    11  Environment and natural resources management
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     7                   Social dev/gender/inclusion
10    5                         Trade and integration
11    4      Financial and private sector development
12    6         Social protection and risk management
13    6         Social protection and risk management
14    2                      Public sector governance
15    4      Financial and private sector development
16   11  Environment and natural resources management
17    8                     